# MLR tests

In here, the goal was to convert the xarray ds to np array ds

In [56]:
import numpy as np
import os
from matplotlib import pyplot as plt
import xarray as xr
import xskillscore as xs
import skimage
import pandas as pd
import seaborn as sns
import math
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/Downscaling_CM/utils"
os.chdir(path)
import metrics
import dataset
import rescaling
import results
import data_preparation
import methods
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
from sklearn.linear_model import LinearRegression

In [41]:
sns.set(style="darkgrid")

In [42]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

Let's get the files

In [5]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/Downscaling_CM/data"
os.chdir(path)
training_ds = xr.open_dataset("training_ds.nc")
coarse_training_ds = xr.open_dataset("coarse_training.nc")

In [6]:
training_ds

<xarray.Dataset>
Dimensions:     (time: 288, rlon: 1542, rlat: 1542)
Coordinates:
  * time        (time) datetime64[ns] 2003-01-01 ... 2003-12-01T23:00:00
  * rlon        (rlon) float32 -18.86 -18.84 -18.82 -18.8 ... 11.92 11.94 11.96
  * rlat        (rlat) float32 -14.86 -14.84 -14.82 -14.8 ... 15.92 15.94 15.96
    lon         (rlat, rlon) float32 ...
    lat         (rlat, rlon) float32 ...
    height_2m   float32 ...
    height_10m  float32 ...
Data variables:
    T_2M        (time, rlat, rlon) float32 ...
    RELHUM_2M   (time, rlat, rlon) float32 ...
    QV_2M       (time, rlat, rlon) float32 ...
    TOT_PR      (time, rlat, rlon) float32 ...
Attributes:
    standard_name:  air_temperature
    long_name:      2m temperature
    units:          K
    grid_mapping:   rotated_pole

## xarray to np.array

In [7]:
training_ds = training_ds.drop('QV_2M')
coarse_training_ds = coarse_training_ds.drop('QV_2M')

In [9]:
training = data_preparation.nc_to_np(training_ds)
coarse_training = data_preparation.nc_to_np(coarse_training_ds)

In [151]:
test

array([[291.18228817, 291.09313872, 291.04156386, ..., 281.31818085,
        281.35640017, 281.39035109],
       [291.19797897, 291.08878306, 291.04109845, ..., 281.37529002,
        281.37254228, 281.43838848],
       [291.15337946, 291.06754049, 291.00719689, ..., 281.36302644,
        281.42729639, 281.47728978],
       ...,
       [282.43142152, 282.28552648, 282.12769019, ..., 244.44181508,
        244.11374132, 244.06910978],
       [282.21159746, 282.11757261, 282.06558331, ..., 243.39847249,
        243.305215  , 243.50755826],
       [282.07549934, 282.04009872, 282.03819709, ..., 242.83023427,
        242.99594971, 243.60281969]])

In [22]:
test = coarse_training[0][0][:-1,:-1]
test[:,0:2]
test[:,0:2].reshape(128,2,2)

array([[[291.18228817, 291.09313872],
        [291.19797897, 291.08878306]],

       [[291.15337946, 291.06754049],
        [291.05648252, 291.01490741]],

       [[291.01586776, 290.96297456],
        [291.02081037, 290.94916367]],

       [[290.99785765, 290.94531655],
        [290.95222397, 290.89898951]],

       [[290.94247709, 290.89696621],
        [290.97021242, 290.9491298 ]],

       [[291.0078422 , 290.99854544],
        [291.03497651, 291.03031648]],

       [[291.05215287, 291.03643283],
        [291.06179299, 291.04882572]],

       [[291.06742349, 291.05571743],
        [291.06552121, 291.0660723 ]],

       [[291.0714774 , 291.0643616 ],
        [291.12310546, 291.08010119]],

       [[291.17370953, 291.13715377],
        [291.20019601, 291.19188905]],

       [[291.22532649, 291.18122817],
        [291.29606359, 291.21101643]],

       [[291.30192656, 291.26849052],
        [291.39664128, 291.34616777]],

       [[291.46843122, 291.42615689],
        [291.46434844, 291

In [50]:
temp = coarse_training[0][:,:-1,:-1]
temp.shape

(288, 256, 256)

In [53]:
lis = list()
lis.append(coarse_training[0][:,:-1,2:4])
lis

[array([[[291.04156386, 290.9799917 ],
         [291.04109845, 290.99388253],
         [291.00719689, 290.93211744],
         ...,
         [282.12769019, 282.09115781],
         [282.06558331, 282.0771875 ],
         [282.03819709, 282.03628557]],
 
        [[291.10905655, 291.05843199],
         [291.10343139, 291.00960104],
         [291.03969786, 290.93212176],
         ...,
         [282.33649948, 282.2176996 ],
         [282.15877757, 282.15976024],
         [282.07385025, 282.08598778]],
 
        [[291.08074523, 291.01898714],
         [291.06951047, 290.96353965],
         [291.02914143, 290.89709637],
         ...,
         [282.47348103, 282.28360736],
         [282.32298986, 282.22996008],
         [282.1954572 , 282.13341745]],
 
        ...,
 
        [[291.34068548, 291.25916636],
         [291.29671555, 291.278152  ],
         [291.30451651, 291.2887633 ],
         ...,
         [283.45768412, 283.47759762],
         [283.39709162, 283.42851447],
         [283.34588626,